# KDF DISSECTOR
## First working complete version in PYTHON

Time taken to convert 1 file so far clocked at ~15 seconds.
Need to optimise EACH step to drop this.
- function calls need to do loops (property builder)
- use lists/indexing to automate searches through all the properties (dictionary of channel lengths: length???!!!!)

### Below this cell are debugging sections to highlight where the 


In [2]:
import pandas as pd
import sys
import os
import numpy as np
import re
import scipy.sparse as sparse
import time

start_time = time.clock()

int=0

for file in os.listdir(os.getcwd()):
    
    if file.endswith(".kdf"):
        int=int+1
        kdf = pd.read_csv(file,names=['Key','Val/Col','Row'])
        
        EOSIndexing = kdf[kdf['Key'].str.contains('<EOS>')]
        SiteIndexing = kdf[kdf['Key'].str.contains('Site_')]

        nosSites = len(SiteIndexing)

        SiteInts = nosSites+1

        Subsite_NumberOf = 4

        # # # # # # #  function to find a TERM at any SITE(n) and SUBSITE(k)  # # # # # # # # # #

        def deviceAndProperties(n,k,term):  
            """
            Returns properties described by 'words' a device (n) and subsite (k)
            """
            SiteChoice = kdf.iloc[range(SiteIndexing.index[n-1],EOSIndexing.index[n-1])]  # choose site
            SubsiteChoice = SiteChoice[SiteChoice['Key'].str.contains('subsite%d' % k)] # choose subsite
            termChoice = SubsiteChoice[SubsiteChoice['Key'].str.contains(term)] # allows search for term
            numpyarray = SubsiteChoice[SubsiteChoice['Key'].str.contains(term)]
            fixedarray = numpyarray['Val/Col'].values ## this is of type numpy array    
            return fixedarray

        def seek(df, *words):  #1
            """
            Return a sub-DataFrame of those rows whose Name column match all the words.
            EXAMPLE USAGE: Mobs1 = seek(kdf, 'OTFT_MOBILITY','subsite1')['Val/Col'].values
            """
            spam = kdf[np.logical_and.reduce([kdf['Key'].str.contains(word) for word in words])]
            return spam['Val/Col'].values

        def propertybuilder(itemAsAString, searchterm):
            item1 = pd.DataFrame(seek(kdf, searchterm, 'subsite1'))
            item2 = pd.DataFrame(seek(kdf, searchterm, 'subsite2'))
            item3 = pd.DataFrame(seek(kdf, searchterm, 'subsite3'))
            item4 = pd.DataFrame(seek(kdf, searchterm, 'subsite4'))
            column = pd.concat([item1, item2, item3, item4], axis=0, ignore_index = True)
            column.columns = [itemAsAString]
            return column

        site1 = SiteIndexing['Key']
        site2 = SiteIndexing['Key']
        site3 = SiteIndexing['Key']
        site4 = SiteIndexing['Key']
        allSites = pd.concat([site1, site2, site3, site4], axis=0, ignore_index = True)

        allLENGTHs = propertybuilder('Channel Lengths', 'LENGTH')
        allWIDTHs = propertybuilder('Channel Widths', 'WIDTH')
        allMAXMOBs = propertybuilder('MaxMobs', 'MAXMOB')
        allIONOFFs = propertybuilder('IOnOff', 'IONOFF')
        allMOBs = propertybuilder('Mobility', 'OTFT_MOBILITY')
        allIOFFs = propertybuilder('IOFF', 'VALID_IOFF2')
        allIONs = propertybuilder('IONs', 'ION@')
        allCAPs = propertybuilder('Caps', 'SPARE2')
        allVDSs = propertybuilder('VDS', 'VDS')
        allVTHs = propertybuilder('VTHs', 'VTH')

        allSingleProps = pd.concat([allSites, allLENGTHs, allWIDTHs, allMAXMOBs, allIONOFFs, allMOBs, 
                                allIOFFs, allIONs, allCAPs, allVDSs, allVTHs], axis=1, ignore_index = False)

        
#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#

        VGateSubsite1 = deviceAndProperties(0,1,'V_Gate')  ## need to convert all these to pd.DataFrames if to 
        VGateSubsite2 = deviceAndProperties(0,2,'V_Gate')  ## use code below to compile into a single column:
        VGateSubsite3 = deviceAndProperties(0,3,'V_Gate')  ## pd.DataFrame(deviceAndProperties(xxxx))
        VGateSubsite4 = deviceAndProperties(0,4,'V_Gate')

        blank = np.zeros((50,36)) ## Generates BLANK ARRAY HERE
        blankdf = pd.DataFrame(blank)

        df_1a = pd.DataFrame()
        df_1b = pd.DataFrame()
        df_2a = pd.DataFrame()
        df_2b = pd.DataFrame()
        df_3a = pd.DataFrame()
        df_3b = pd.DataFrame()
        df_4a = pd.DataFrame()
        df_4b = pd.DataFrame()


        df_1a['VGates SS1'] = VGateSubsite1    ## --- STILL NEED TO CALCULATE MODAL VALUE FOR THIS!!!!
        for i in range(1,nosSites+1):
            IDrain_Subsite1 = deviceAndProperties(i,1,'I_Drain')
            df_1a[(i-1)]= IDrain_Subsite1
            Site1a = df_1a.combine_first(blankdf)

        for i in range(1,nosSites+1):
            IGate_Subsite1 = deviceAndProperties(i,1,'I_Gate')
            df_1b[(i-1)]= IGate_Subsite1
            Site1b = df_1b.combine_first(blankdf)

        #------------------------------------------------------------------------------------------------#

        df_2a['VGates SS2'] = VGateSubsite2    ## --- STILL NEED TO CALCULATE MODAL VALUE FOR THIS!!!!
        for i in range(1,nosSites+1):
            IDrain_Subsite2 = deviceAndProperties(i,2,'I_Drain')
            df_2a[(i-1)]= IDrain_Subsite2
            Site2a = df_2a.combine_first(blankdf)

        for i in range(1,nosSites+1):
            IGate_Subsite2 = deviceAndProperties(i,2,'I_Gate')
            df_2b[(i-1)]= IGate_Subsite2
            Site2b = df_2b.combine_first(blankdf)

        #------------------------------------------------------------------------------------------------#

        df_3a['VGates SS3'] = VGateSubsite3    ## --- STILL NEED TO CALCULATE MODAL VALUE FOR THIS!!!!
        for i in range(1,nosSites+1):
            IDrain_Subsite3 = deviceAndProperties(i,3,'I_Drain')
            df_3a[(i-1)]= IDrain_Subsite3
            Site3a = df_3a.combine_first(blankdf)

        for i in range(1,nosSites+1):
            IGate_Subsite3 = deviceAndProperties(i,3,'I_Gate')
            df_3b[(i-1)]= IGate_Subsite3
            Site3b = df_3b.combine_first(blankdf)

        #------------------------------------------------------------------------------------------------#

        df_4a['VGates SS4'] = VGateSubsite4    ## --- STILL NEED TO CALCULATE MODAL VALUE FOR THIS!!!!
        for i in range(1,nosSites+1):
            IDrain_Subsite4 = deviceAndProperties(i,4,'I_Drain')
            df_4a[(i-1)]= IDrain_Subsite4
            Site4a = df_4a.combine_first(blankdf)

        for i in range(1,nosSites+1):
            IGate_Subsite4 = deviceAndProperties(i,4,'I_Gate')
            df_4b[(i-1)]= IGate_Subsite4
            Site4b = df_4b.combine_first(blankdf)

        ## Will need to use concat method on the pd object such as
        result = pd.concat([Site1a, Site1b, Site2a, Site2b, Site3a, Site3b, Site4a, Site4b, allSingleProps], 
                           axis=1, join='inner', ignore_index = True)

        total_time = ((time.clock() - start_time))
        print("--- %s seconds ---" % round(total_time,2))
        
        Output = os.path.splitext(file)[0]+'.csv'
        result.to_csv(Output, index=False, header = False)
        
    else:
        continue
print(int)

C:\Users\ben.coombs\Local_Docs\WinPython\python-3.4.3\lib\site-packages\pandas\core\index.py:2271: RuntimeWarning: unorderable types: str() < int(), sort order is undefined for incomparable objects
  return_indexers=return_indexers)
C:\Users\ben.coombs\Local_Docs\WinPython\python-3.4.3\lib\site-packages\pandas\core\index.py:2271: RuntimeWarning: unorderable types: int() > str(), sort order is undefined for incomparable objects
  return_indexers=return_indexers)


--- 16.09 seconds ---
--- 31.89 seconds ---
--- 49.16 seconds ---
--- 66.05 seconds ---
4


# dissector broken down to just 'single' parameters...

In [1]:
import pandas as pd
import sys
import os
import numpy as np
import re
import scipy.sparse as sparse
import time

start_time = time.clock()

int=0

for file in os.listdir(os.getcwd()):
    
    if file.endswith(".kdf"):
        int=int+1
        kdf = pd.read_csv(file,names=['Key','Val/Col','Row'])
        
        EOSIndexing = kdf[kdf['Key'].str.contains('<EOS>')]
        SiteIndexing = kdf[kdf['Key'].str.contains('Site_')]

        nosSites = len(SiteIndexing)

        SiteInts = nosSites+1

        Subsite_NumberOf = 4

        # # # # # # # #  function to find a TERM at any SITE(n) and SUBSITE(k)  # # # # # # # # # # # #

        def deviceAndProperties(n,k,term):  
            """
            Returns properties described by 'words' a device (n) and subsite (k)
            """
            SiteChoice = kdf.iloc[range(SiteIndexing.index[n-1],EOSIndexing.index[n-1])]  # choose site
            SubsiteChoice = SiteChoice[SiteChoice['Key'].str.contains('subsite%d' % k)] # choose subsite
            termChoice = SubsiteChoice[SubsiteChoice['Key'].str.contains(term)] # allows search for term
            numpyarray = SubsiteChoice[SubsiteChoice['Key'].str.contains(term)]
            fixedarray = numpyarray['Val/Col'].values ## this is of type numpy array    
            return fixedarray

        def seek(df, *words):  #1
            """
            Return a sub-DataFrame of those rows whose Name column match all the words.
            EXAMPLE USAGE: Mobs1 = seek(kdf, 'OTFT_MOBILITY','subsite1')['Val/Col'].values
            """
            spam = kdf[np.logical_and.reduce([kdf['Key'].str.contains(word) for word in words])]
            return spam['Val/Col'].values

        def propertybuilder(itemAsAString, searchterm):
            item1 = pd.DataFrame(seek(kdf, searchterm, 'subsite1'))
            item2 = pd.DataFrame(seek(kdf, searchterm, 'subsite2'))
            item3 = pd.DataFrame(seek(kdf, searchterm, 'subsite3'))
            item4 = pd.DataFrame(seek(kdf, searchterm, 'subsite4'))
            column = pd.concat([item1, item2, item3, item4], axis=0, ignore_index = True)
            column.columns = [itemAsAString]
            return column

        site1 = SiteIndexing['Key']
        site2 = SiteIndexing['Key']
        site3 = SiteIndexing['Key']
        site4 = SiteIndexing['Key']
        allSites = pd.concat([site1, site2, site3, site4], axis=0, ignore_index = True)

        allLENGTHs = propertybuilder('Channel Lengths', 'LENGTH')
        allWIDTHs = propertybuilder('Channel Widths', 'WIDTH')
        allMAXMOBs = propertybuilder('MaxMobs', 'MAXMOB')
        allIONOFFs = propertybuilder('IOnOff', 'IONOFF')
        allMOBs = propertybuilder('Mobility', 'OTFT_MOBILITY')
        allIOFFs = propertybuilder('IOFF', 'VALID_IOFF2')
        allIONs = propertybuilder('IONs', 'ION@')
        allCAPs = propertybuilder('Caps', 'SPARE2')
        allVDSs = propertybuilder('VDS', 'VDS')
        allVTHs = propertybuilder('VTHs', 'VTH')

        allSingleProps = pd.concat([allSites, allLENGTHs, allWIDTHs, allMAXMOBs, allIONOFFs, allMOBs, 
                                allIOFFs, allIONs, allCAPs, allVDSs, allVTHs], axis=1, ignore_index = False)

#         VGateSubsite1 = deviceAndProperties(0,1,'V_Gate')  ## need to convert all these to pd.DataFrames if to 
#         VGateSubsite2 = deviceAndProperties(0,2,'V_Gate')  ## use code below to compile into a single column:
#         VGateSubsite3 = deviceAndProperties(0,3,'V_Gate')  ## pd.DataFrame(deviceAndProperties(xxxx))
#         VGateSubsite4 = deviceAndProperties(0,4,'V_Gate')

#         blank = np.zeros((50,36)) ## Generates BLANK ARRAY HERE
#         blankdf = pd.DataFrame(blank)

#         df_1a = pd.DataFrame()
#         df_1b = pd.DataFrame()
#         df_2a = pd.DataFrame()
#         df_2b = pd.DataFrame()
#         df_3a = pd.DataFrame()
#         df_3b = pd.DataFrame()
#         df_4a = pd.DataFrame()
#         df_4b = pd.DataFrame()


#         df_1a['VGates SS1'] = VGateSubsite1    ## --- STILL NEED TO CALCULATE MODAL VALUE FOR THIS!!!!
#         for i in range(1,nosSites+1):
#             IDrain_Subsite1 = deviceAndProperties(i,1,'I_Drain')
#             df_1a[(i-1)]= IDrain_Subsite1
#             Site1a = df_1a.combine_first(blankdf)

#         for i in range(1,nosSites+1):
#             IGate_Subsite1 = deviceAndProperties(i,1,'I_Gate')
#             df_1b[(i-1)]= IGate_Subsite1
#             Site1b = df_1b.combine_first(blankdf)

#         #------------------------------------------------------------------------------------------------#

#         df_2a['VGates SS2'] = VGateSubsite2    ## --- STILL NEED TO CALCULATE MODAL VALUE FOR THIS!!!!
#         for i in range(1,nosSites+1):
#             IDrain_Subsite2 = deviceAndProperties(i,2,'I_Drain')
#             df_2a[(i-1)]= IDrain_Subsite2
#             Site2a = df_2a.combine_first(blankdf)

#         for i in range(1,nosSites+1):
#             IGate_Subsite2 = deviceAndProperties(i,2,'I_Gate')
#             df_2b[(i-1)]= IGate_Subsite2
#             Site2b = df_2b.combine_first(blankdf)

#         #------------------------------------------------------------------------------------------------#

#         df_3a['VGates SS3'] = VGateSubsite3    ## --- STILL NEED TO CALCULATE MODAL VALUE FOR THIS!!!!
#         for i in range(1,nosSites+1):
#             IDrain_Subsite3 = deviceAndProperties(i,3,'I_Drain')
#             df_3a[(i-1)]= IDrain_Subsite3
#             Site3a = df_3a.combine_first(blankdf)

#         for i in range(1,nosSites+1):
#             IGate_Subsite3 = deviceAndProperties(i,3,'I_Gate')
#             df_3b[(i-1)]= IGate_Subsite3
#             Site3b = df_3b.combine_first(blankdf)

#         #------------------------------------------------------------------------------------------------#

#         df_4a['VGates SS4'] = VGateSubsite4    ## --- STILL NEED TO CALCULATE MODAL VALUE FOR THIS!!!!
#         for i in range(1,nosSites+1):
#             IDrain_Subsite4 = deviceAndProperties(i,4,'I_Drain')
#             df_4a[(i-1)]= IDrain_Subsite4
#             Site4a = df_4a.combine_first(blankdf)

#         for i in range(1,nosSites+1):
#             IGate_Subsite4 = deviceAndProperties(i,4,'I_Gate')
#             df_4b[(i-1)]= IGate_Subsite4
#             Site4b = df_4b.combine_first(blankdf)

#         ## Will need to use concat method on the pd object such as
#         result = pd.concat([Site1a, Site1b, Site2a, Site2b, Site3a, Site3b, Site4a, Site4b, allSingleProps], 
#                            axis=1, join='inner', ignore_index = True)

        result = allSingleProps
    
        total_time = ((time.clock() - start_time))
        print("--- %s seconds ---" % round(total_time,2))
        
        Output = os.path.splitext(file)[0]+'SINGLES.csv'
        result.to_csv(Output, index=False, header = False)
        
    else:
        continue
print(int)

--- 2.78 seconds ---
--- 6.22 seconds ---
--- 11.12 seconds ---
--- 13.92 seconds ---
4


In [3]:
import pandas as pd
import sys
import os
import numpy as np
import re
import scipy.sparse as sparse
import time

start_time = time.clock()

int=0

for file in os.listdir(os.getcwd()):
    
    if file.endswith(".kdf"):
        int=int+1
        kdf = pd.read_csv(file,names=['Key','Val/Col','Row'])
        
        EOSIndexing = kdf[kdf['Key'].str.contains('<EOS>')]
        SiteIndexing = kdf[kdf['Key'].str.contains('Site_')]

        nosSites = len(SiteIndexing)

        SiteInts = nosSites+1

        Subsite_NumberOf = 4

        # # # # # # # #  function to find a TERM at any SITE(n) and SUBSITE(k)  # # # # # # # # # # # #

        def deviceAndProperties(n,k,term):  
            """
            Returns properties described by 'words' a device (n) and subsite (k)
            """
            SiteChoice = kdf.iloc[range(SiteIndexing.index[n-1],EOSIndexing.index[n-1])]  # choose site
            SubsiteChoice = SiteChoice[SiteChoice['Key'].str.contains('subsite%d' % k)] # choose subsite
            termChoice = SubsiteChoice[SubsiteChoice['Key'].str.contains(term)] # allows search for term
            numpyarray = SubsiteChoice[SubsiteChoice['Key'].str.contains(term)]
            fixedarray = numpyarray['Val/Col'].values ## this is of type numpy array    
            return fixedarray

        def seek(df, *words):  #1
            """
            Return a sub-DataFrame of those rows whose Name column match all the words.
            EXAMPLE USAGE: Mobs1 = seek(kdf, 'OTFT_MOBILITY','subsite1')['Val/Col'].values
            """
            spam = kdf[np.logical_and.reduce([kdf['Key'].str.contains(word) for word in words])]
            return spam['Val/Col'].values

        def propertybuilder(itemAsAString, searchterm):
            item1 = pd.DataFrame(seek(kdf, searchterm, 'subsite1'))
            item2 = pd.DataFrame(seek(kdf, searchterm, 'subsite2'))
            item3 = pd.DataFrame(seek(kdf, searchterm, 'subsite3'))
            item4 = pd.DataFrame(seek(kdf, searchterm, 'subsite4'))
            column = pd.concat([item1, item2, item3, item4], axis=0, ignore_index = True)
            column.columns = [itemAsAString]
            return column

#         site1 = SiteIndexing['Key']
#         site2 = SiteIndexing['Key']
#         site3 = SiteIndexing['Key']
#         site4 = SiteIndexing['Key']
#         allSites = pd.concat([site1, site2, site3, site4], axis=0, ignore_index = True)

#         allLENGTHs = propertybuilder('Channel Lengths', 'LENGTH')
#         allWIDTHs = propertybuilder('Channel Widths', 'WIDTH')
#         allMAXMOBs = propertybuilder('MaxMobs', 'MAXMOB')
#         allIONOFFs = propertybuilder('IOnOff', 'IONOFF')
#         allMOBs = propertybuilder('Mobility', 'OTFT_MOBILITY')
#         allIOFFs = propertybuilder('IOFF', 'VALID_IOFF2')
#         allIONs = propertybuilder('IONs', 'ION@')
#         allCAPs = propertybuilder('Caps', 'SPARE2')
#         allVDSs = propertybuilder('VDS', 'VDS')
#         allVTHs = propertybuilder('VTHs', 'VTH')

#         allSingleProps = pd.concat([allSites, allLENGTHs, allWIDTHs, allMAXMOBs, allIONOFFs, allMOBs, 
#                                 allIOFFs, allIONs, allCAPs, allVDSs, allVTHs], axis=1, ignore_index = False)

        VGateSubsite1 = deviceAndProperties(0,1,'V_Gate')  ## need to convert all these to pd.DataFrames if to 
        VGateSubsite2 = deviceAndProperties(0,2,'V_Gate')  ## use code below to compile into a single column:
        VGateSubsite3 = deviceAndProperties(0,3,'V_Gate')  ## pd.DataFrame(deviceAndProperties(xxxx))
        VGateSubsite4 = deviceAndProperties(0,4,'V_Gate')

        blank = np.zeros((50,36)) ## Generates BLANK ARRAY HERE
        blankdf = pd.DataFrame(blank)

        df_1a = pd.DataFrame()
        df_1b = pd.DataFrame()
        df_2a = pd.DataFrame()
        df_2b = pd.DataFrame()
        df_3a = pd.DataFrame()
        df_3b = pd.DataFrame()
        df_4a = pd.DataFrame()
        df_4b = pd.DataFrame()


        df_1a['VGates SS1'] = VGateSubsite1    ## --- STILL NEED TO CALCULATE MODAL VALUE FOR THIS!!!!
        for i in range(1,nosSites+1):
            IDrain_Subsite1 = deviceAndProperties(i,1,'I_Drain')
            df_1a[(i-1)]= IDrain_Subsite1
            Site1a = df_1a.combine_first(blankdf)

        for i in range(1,nosSites+1):
            IGate_Subsite1 = deviceAndProperties(i,1,'I_Gate')
            df_1b[(i-1)]= IGate_Subsite1
            Site1b = df_1b.combine_first(blankdf)

        #------------------------------------------------------------------------------------------------#

        df_2a['VGates SS2'] = VGateSubsite2    ## --- STILL NEED TO CALCULATE MODAL VALUE FOR THIS!!!!
        for i in range(1,nosSites+1):
            IDrain_Subsite2 = deviceAndProperties(i,2,'I_Drain')
            df_2a[(i-1)]= IDrain_Subsite2
            Site2a = df_2a.combine_first(blankdf)

        for i in range(1,nosSites+1):
            IGate_Subsite2 = deviceAndProperties(i,2,'I_Gate')
            df_2b[(i-1)]= IGate_Subsite2
            Site2b = df_2b.combine_first(blankdf)

        #------------------------------------------------------------------------------------------------#

        df_3a['VGates SS3'] = VGateSubsite3    ## --- STILL NEED TO CALCULATE MODAL VALUE FOR THIS!!!!
        for i in range(1,nosSites+1):
            IDrain_Subsite3 = deviceAndProperties(i,3,'I_Drain')
            df_3a[(i-1)]= IDrain_Subsite3
            Site3a = df_3a.combine_first(blankdf)

        for i in range(1,nosSites+1):
            IGate_Subsite3 = deviceAndProperties(i,3,'I_Gate')
            df_3b[(i-1)]= IGate_Subsite3
            Site3b = df_3b.combine_first(blankdf)

        #------------------------------------------------------------------------------------------------#

        df_4a['VGates SS4'] = VGateSubsite4    ## --- STILL NEED TO CALCULATE MODAL VALUE FOR THIS!!!!
        for i in range(1,nosSites+1):
            IDrain_Subsite4 = deviceAndProperties(i,4,'I_Drain')
            df_4a[(i-1)]= IDrain_Subsite4
            Site4a = df_4a.combine_first(blankdf)

        for i in range(1,nosSites+1):
            IGate_Subsite4 = deviceAndProperties(i,4,'I_Gate')
            df_4b[(i-1)]= IGate_Subsite4
            Site4b = df_4b.combine_first(blankdf)

        ## Will need to use concat method on the pd object such as
        result = pd.concat([Site1a, Site1b, Site2a, Site2b, Site3a, Site3b, Site4a, Site4b], 
                           axis=1, join='inner', ignore_index = True)

        total_time = ((time.clock() - start_time))
        print("--- %s seconds ---" % round(total_time,2))
        
        Output = os.path.splitext(file)[0]+'MULTI.csv'
        result.to_csv(Output, index=False, header = False)
        
    else:
        continue
print(int)

C:\Users\ben.coombs\Local_Docs\WinPython\python-3.4.3\lib\site-packages\pandas\core\index.py:2271: RuntimeWarning: unorderable types: str() < int(), sort order is undefined for incomparable objects
  return_indexers=return_indexers)
C:\Users\ben.coombs\Local_Docs\WinPython\python-3.4.3\lib\site-packages\pandas\core\index.py:2271: RuntimeWarning: unorderable types: int() > str(), sort order is undefined for incomparable objects
  return_indexers=return_indexers)


--- 12.66 seconds ---
--- 25.51 seconds ---
--- 38.22 seconds ---
--- 51.14 seconds ---
4
